In [14]:
import pandas as pd
import numpy as np

In [97]:
#u_data-> dataframe으로 변경
u_data = []
with open('./ml-100k/u.data','r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_data.append(line.strip().split('\t'))
    
u_data = pd.DataFrame(u_data, columns=["userID","itemID","rating","timestamp"])  
u_data.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [104]:
#u.item -> dataframe으로 변경
u_item = []
with open("./ml-100k/u.item",'r',encoding='ISO-8859-1') as f:
    for line in f.readlines():
        u_item.append(line.strip().split("|")[:-4]) #README에서 item의 컬럼은 20개 였으나 해당 데이터에는 24개의 컬럼이 존재하여 4개의 컬럼을 제외시켰습니다
        

        
movies_info = """movie id | movie title | release date | video release date |
IMDb URL | unknown | Action | Adventure | Animation |
Children's | Comedy | Crime | Documentary | Drama | Fantasy |
Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
Thriller | War | Western"""        
        
movies_info = movies_info.split(" | ")    

u_item = pd.DataFrame(u_item,columns=movies_info)
u_item

,movie id,movie title,release date,video release date |\nIMDb URL,unknown,Action,Adventure,Animation |\nChildren's,Comedy,Crime,Documentary,Drama,Fantasy |\nFilm-Noir,Horror,Musical,Mystery,Romance,Sci-Fi |\nThriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Four Rooms (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Get Shorty (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1679,1680,Sliding Doors (1998),01-Jan-1998,,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1680,1681,You So Crazy (1994),01-Jan-1994,,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [99]:
#u.user -> dataframe으로 변경
u_user = []
with open("./ml-100k/u.user",'r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_user.append(line.strip().split("|")) 
        
user_info = """user id | age | gender | occupation | zip code"""        
        
user_info = user_info.split(" | ")    

u_user = pd.DataFrame(u_user,columns=user_info)
u_user

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [100]:
#u.genre ->dataframe으로 변경
genre=[]
with open("./ml-100k/u.genre",'r',encoding='UTF-8') as f:
    for line in f.readlines():
        genre.append(line.strip().split("|")) 
genre = pd.DataFrame(genre,columns=["genre_name","genre_ID"]) #어차피 Index와 Genre_ID가 같음 나중에 없애봐
print(genre)

#u.occupation-> dataframe으로 변경
occupation =[]
with open("./ml-100k/u.occupation", 'r',encoding='UTF-8') as f:
    for line in f.readlines():
        occupation.append(line.strip().split("|"))
        
occupation = pd.DataFrame(occupation,columns=["job"])
occupation

     genre_name genre_ID
0       unknown        0
1        Action        1
2     Adventure        2
3     Animation        3
4    Children's        4
5        Comedy        5
6         Crime        6
7   Documentary        7
8         Drama        8
9       Fantasy        9
10    Film-Noir       10
11       Horror       11
12      Musical       12
13      Mystery       13
14      Romance       14
15       Sci-Fi       15
16     Thriller       16
17          War       17
18      Western       18
19                  None


,job
0,administrator
1,artist
2,doctor
3,educator
4,engineer
5,entertainment
6,executive
7,healthcare
8,homemaker
9,lawyer


In [134]:
# 1.전체이용자에서 가장 시청률이 높은 영화 TOP 100
result = u_data.value_counts(subset="itemID", ascending=False)

result = pd.DataFrame(result)
top_id = result.index[:100] #영화 TOP 100 ID

print("1. 전체 사용자에서 가장 시청률이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==top_id[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))


1. 전체 사용자에서 가장 시청률이 높은 영화 TOP 100

1등 영화 제목 : Star Wars (1977)
2등 영화 제목 : Contact (1997)
3등 영화 제목 : Fargo (1996)
4등 영화 제목 : Return of the Jedi (1983)
5등 영화 제목 : Liar Liar (1997)
6등 영화 제목 : English Patient, The (1996)
7등 영화 제목 : Scream (1996)
8등 영화 제목 : Toy Story (1995)
9등 영화 제목 : Air Force One (1997)
10등 영화 제목 : Independence Day (ID4) (1996)
11등 영화 제목 : Raiders of the Lost Ark (1981)
12등 영화 제목 : Godfather, The (1972)
13등 영화 제목 : Pulp Fiction (1994)
14등 영화 제목 : Twelve Monkeys (1995)
15등 영화 제목 : Silence of the Lambs, The (1991)
16등 영화 제목 : Jerry Maguire (1996)
17등 영화 제목 : Rock, The (1996)
18등 영화 제목 : Empire Strikes Back, The (1980)
19등 영화 제목 : Star Trek: First Contact (1996)
20등 영화 제목 : Back to the Future (1985)
21등 영화 제목 : Titanic (1997)
22등 영화 제목 : Mission: Impossible (1996)
23등 영화 제목 : Fugitive, The (1993)
24등 영화 제목 : Indiana Jones and the Last Crusade (1989)
25등 영화 제목 : Willy Wonka and the Chocolate Factory (1971)
26등 영화 제목 : Princess Bride, The (1987)
27등 영화 제목 : Forrest Gump (1994)

In [113]:
#u_item[u_item['itemID'] == '71']
u_item.loc[u_item['movie id']=='1',"movie title"]

0    Toy Story (1995)
Name: movie title, dtype: object